## Classification with LogisticRegression

LogisticRegression is a classifier part of linear model package. It is used for classifying the dataset with a binary class.


In [2]:
# Import the required libraries
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.linear_model import LogisticRegression


# Reading data from the csv
titanic_ds = pd.read_csv("../datasets/titanic_train.csv")

titanic_ds.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
# Getting the predictors
titanic_ds.columns
predictors = ['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Embarked']

X = titanic_ds[predictors]
y = titanic_ds.Survived

# Converting categorical values to int
X['Sex'] = X['Sex'].astype('category')
X['Embarked'] = X['Embarked'].astype('category')

category_cols = X.select_dtypes(['category']).columns
category_cols

D:\space\kepler\programs\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\space\kepler\programs\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Index(['Sex', 'Embarked'], dtype='object')

In [36]:
X[category_cols] = X[category_cols].apply(lambda x: x.cat.codes)
X

D:\space\kepler\programs\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:2378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


,PassengerId,Pclass,Sex,Age,SibSp,Embarked
0,1,3,1,22.0,1,2
1,2,1,0,38.0,1,0
2,3,3,0,26.0,0,2
3,4,1,0,35.0,1,2
4,5,3,1,35.0,0,2
5,6,3,1,NaN,0,1
6,7,1,1,54.0,0,2
7,8,3,1,2.0,3,2
8,9,3,0,27.0,0,2
9,10,2,0,14.0,1,0


In [37]:
# Function to impute the age values
def impute_age(cols):
    pclass = cols[0]
    age = cols[1]
    
    if pd.isnull(age):
        if pclass == 1:
            return 50
        elif pclass == 2:
            return 35
        else:
            return 20
    else:
        return age
    
X[['Pclass', 'Age']]

,Pclass,Age
0,3,22.0
1,1,38.0
2,3,26.0
3,1,35.0
4,3,35.0
5,3,NaN
6,1,54.0
7,3,2.0
8,3,27.0
9,2,14.0


In [38]:
X['Age'] = X[['Pclass', 'Age']].apply(impute_age, axis=1)

X

D:\space\kepler\programs\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,PassengerId,Pclass,Sex,Age,SibSp,Embarked
0,1,3,1,22.0,1,2
1,2,1,0,38.0,1,0
2,3,3,0,26.0,0,2
3,4,1,0,35.0,1,2
4,5,3,1,35.0,0,2
5,6,3,1,20.0,0,1
6,7,1,1,54.0,0,2
7,8,3,1,2.0,3,2
8,9,3,0,27.0,0,2
9,10,2,0,14.0,1,0
